In [10]:
pip install tensorflow

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
from matplotlib.pyplot import figure
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv('aapl_data.csv')

In [3]:
# Preprocessing
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.date
df.set_index('time', inplace=True)
# Sort the DataFrame by date
df = df.sort_index(ascending=True)
df = df[df['volume'] != 0]  # Remove rows where volume is 0

In [6]:
# Calculate future returns and signals
df['future_returns'] = df['close'].pct_change().shift(-1)
signal = []
for i in range(len(df)):
    if df['future_returns'].iloc[i] > 0:
        signal.append(1)
    elif df['future_returns'].iloc[i] < 0:
        signal.append(2)
    else:
        signal.append(3)

df['signal'] = signal

# Calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['close'], 14)

# Calculate ADX
def calculate_adx(df, period=50):
    high = df['high']
    low = df['low']
    close = df['close']

    plus_dm = high.diff()
    minus_dm = low.diff()

    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    atr = true_range.rolling(window=period).mean()

    plus_di = 100 * (plus_dm.rolling(window=period).mean() / atr)
    minus_di = 100 * (minus_dm.abs().rolling(window=period).mean() / atr)

    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.rolling(window=period).mean()
    return adx

df['adx'] = calculate_adx(df, 50)

# Calculate NATR
def calculate_natr(df, period=50):
    tr1 = df['high'] - df['low']
    tr2 = abs(df['high'] - df['close'].shift(1))
    tr3 = abs(df['low'] - df['close'].shift(1))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(window=period).mean()
    natr = (atr / df['close']) * 100
    return natr

df['NATR'] = calculate_natr(df, 50)

# Calculate SMA
df['sma'] = df['close'].rolling(window=30).mean()

# Calculate correlation
df['corr'] = df['close'].rolling(window=int(6.5 * 4)).corr(df['sma'])

# Other features
df['Open-Close'] = (df['open'] - df['close']).shift(-1)
df['Close-High'] = (df['close'] - df['high']).shift(-1)
df['Close-Low'] = (df['close'] - df['low']).shift(-1)

# Drop NaN values
df.dropna(inplace=True)

# Define target and features
y = df[['signal']].copy()
X = df[['rsi', 'adx', 'corr', 'sma', 'NATR', 'volume', 'Open-Close', 'Close-Low', 'Close-High']].copy()

In [8]:
# Split data into training and testing sets
split_percentage = 0.7
split = int(split_percentage * len(df))

X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
# One-hot encode the target variable
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Build the ANN model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(y_train_encoded.shape[1], activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_encoded,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.5368 - loss: 0.9728 - val_accuracy: 0.7863 - val_loss: 0.4971
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7747 - loss: 0.5550 - val_accuracy: 0.8030 - val_loss: 0.4783
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7895 - loss: 0.4904 - val_accuracy: 0.8097 - val_loss: 0.4784
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8124 - loss: 0.4461 - val_accuracy: 0.8114 - val_loss: 0.4593
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8012 - loss: 0.4485 - val_accuracy: 0.8097 - val_loss: 0.4521
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8221 - loss: 0.4094 - val_accuracy: 0.8030 - val_loss: 0.4584
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8128 - loss: 0.4339 - val_accuracy: 0.8030 - val_loss: 0.4474
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8309 - loss: 0.4075 - val_accuracy: 0.8030 - val_l

In [15]:
# Predict on test data
y_pred_encoded = model.predict(X_test)
y_pred = encoder.inverse_transform(y_pred_encoded)

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [24]:
# Evaluate the model
Accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {Accuracy}")

Accuracy: 0.7918939984411536


In [17]:
# Classification report
classification_report_data = classification_report(y_test, y_pred)
print(classification_report_data)

              precision    recall  f1-score   support

           1       0.84      0.76      0.80       686
           2       0.75      0.84      0.79       594
           3       0.00      0.00      0.00         3

    accuracy                           0.79      1283
   macro avg       0.53      0.53      0.53      1283
weighted avg       0.80      0.79      0.79      1283



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
# Make a prediction on the last row of X
Prediction_encoded = model.predict(X_test)[-1]  # Predict using the ANN model
Prediction = encoder.inverse_transform([Prediction_encoded])  # Decode the one-hot prediction to original label

# Translate the prediction to actionable signals
if Prediction[0] == 1:
    print('Buy')
elif Prediction[0] == 2:
    print('Sell')
else:
    print('Hold')
print(Prediction[0])


41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Sell
[2]


In [21]:
split_percentage = 0.7
split = int(split_percentage*len(df))
temp=pd.DataFrame()
temp['close'] = df.close.iloc[split:]
temp['signal']=np.array(y_pred)
print(temp.tail())

                 close  signal
time                          
2024-10-25  231.410004       1
2024-10-28  233.399994       1
2024-10-29  233.669998       2
2024-10-30  230.100006       2
2024-10-31  225.910004       2


In [22]:
# Variable Required for decision making
buy=[]
sell=[]
profit=[]
Close_profit=[]
Stop_loss=[]
Close_order_buy=[]
Close_order_sell=[]
position_Buy=None
position_Sell=None
profit=0.4
lose=0.4

In [25]:
for i in range(len(temp)):
   #####################################################################################################
    if position_Buy!="Buy"  and position_Sell!="Sell" and Accuracy >0.50 and temp['signal'][i]== 1 :                          #Buy Position
                    buy.append(temp['close'][i])
                    sell.append(np.nan)
                    Close_profit.append(np.nan)
                    Stop_loss.append(np.nan)
                    Close_order_buy.append(temp['close'][i])
                    position_Buy="Buy"
    #############################################################################################
    elif  position_Sell!="Sell"  and position_Buy!="Buy" and Accuracy >0.50 and temp['signal'][i]== 2   :                     #Sell Position
                sell.append(temp['close'][i])
                buy.append(np.nan)
                Close_profit.append(np.nan)
                Stop_loss.append(np.nan)
                Close_order_sell.append(temp['close'][i])
                position_Sell="Sell"

    #################################################################################################
    elif position_Sell=="Sell" :                                               #Closing Order  After  Buying a trade(Profit)
        if ((Close_order_sell[-1]+lose) <= temp['close'][i]):                   #Take Profit Condition
            Close_profit.append(np.nan)                                        #append to take profit
            Stop_loss.append(temp['close'][i])
            buy.append(np.nan)
            sell.append(np.nan)
            position_Sell=None
        elif ((Close_order_sell[-1]-(profit)) >= temp['close'][i]):                   #Stop Loss Condition
            Close_profit.append(temp['close'][i])
            Stop_loss.append(np.nan)                                                #Appending to stop loss
            buy.append(np.nan)
            sell.append(np.nan)
            position_Sell=None
        else:
            Close_profit.append(np.nan)
            Stop_loss.append(np.nan)
            buy.append(np.nan)
            sell.append(np.nan)
    ########################################################################################################
    elif position_Buy=="Buy" :                                               #Closing Order  After  Buying a trade(Profit)
        if ((Close_order_buy[-1]+profit) <= temp['close'][i]):                  #Take Profit Condition
            Close_profit.append(temp['close'][i])                          #append to take profit
            Stop_loss.append(np.nan)
            buy.append(np.nan)
            sell.append(np.nan)
            position_Buy=None

        elif ((Close_order_buy[-1]-lose) >= temp['close'][i]):                   #Stop Loss Condition
            Close_profit.append(np.nan)
            Stop_loss.append(temp['close'][i])                                  #Appending to stop loss
            buy.append(np.nan)
            sell.append(np.nan)
            position_Buy=None

        else:
            Close_profit.append(np.nan)
            Stop_loss.append(np.nan)
            buy.append(np.nan)
            sell.append(np.nan)
#################################################################
    else:
            buy.append(np.nan)
            sell.append(np.nan)
            Stop_loss.append(np.nan)
            Close_profit.append(np.nan)
temp['buy']=buy
temp['sell']=sell
temp['Close_profit']=Close_profit
temp['Stop_loss']=Stop_loss
print((temp[temp['buy']!=np.nan]).count())
print(f"Profitibily { temp['Close_profit'].count()/(temp['Close_profit'].count()+temp['Stop_loss'].count())}")

close           1283
signal          1283
buy              292
sell             292
Close_profit     461
Stop_loss        122
dtype: int64
Profitibily 0.79073756432247


<ipython-input-25-708f1d65bd04>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if position_Buy!="Buy"  and position_Sell!="Sell" and Accuracy >0.50 and temp['signal'][i]== 1 :                          #Buy Position
<ipython-input-25-708f1d65bd04>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  buy.append(temp['close'][i])
<ipython-input-25-708f1d65bd04>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Close_order_buy.append(temp[